In [2]:
# import os
# numbers_to_remove = [
#     4, 6, 7, 8, 9, 19, 20, 21, 22, 36, 38, 46, 47, 48, 50, 52, 55, 60, 62, 65,
#     66, 69, 70, 72, 75, 76, 77, 78, 79, 80, 82, 84, 87, 88, 90, 91, 94, 95,
#     96, 97, 102, 112, 113, 114, 117, 118, 119, 120, 121, 123, 124, 126, 127,
#     132, 136, 138, 140, 145, 151, 152, 154, 155, 158, 159, 160, 163, 164,
#     165, 167, 168, 169, 170, 174, 176, 178, 179, 180, 181, 182, 184, 185,
#     187, 188, 189, 190, 191, 192, 195, 196, 197, 198, 199, 200, 202, 207, 208
# ]
#
# files_to_remove = []
#
#
# path = "./data/train/fainted"
# for i in numbers_to_remove:
#     if i < 10:
#         file_name = f"fall00{i}.jpg"
#     elif i < 100:
#         file_name = f"fall0{i}.jpg"
#     else:
#         file_name = f"fall{i}.jpg"
#
#     file_path = path + "/" + file_name
#     try:
#         os.remove(file_path)
#         print(f"{file_path} removed")
#     except FileNotFoundError as e:
#         print(e)





./data/train/fainted/fall004.jpg removed
./data/train/fainted/fall006.jpg removed
./data/train/fainted/fall007.jpg removed
./data/train/fainted/fall008.jpg removed
./data/train/fainted/fall009.jpg removed
./data/train/fainted/fall019.jpg removed
./data/train/fainted/fall020.jpg removed
./data/train/fainted/fall021.jpg removed
./data/train/fainted/fall022.jpg removed
./data/train/fainted/fall036.jpg removed
./data/train/fainted/fall038.jpg removed
./data/train/fainted/fall046.jpg removed
./data/train/fainted/fall047.jpg removed
./data/train/fainted/fall048.jpg removed
./data/train/fainted/fall050.jpg removed
./data/train/fainted/fall052.jpg removed
./data/train/fainted/fall055.jpg removed
./data/train/fainted/fall060.jpg removed
./data/train/fainted/fall062.jpg removed
./data/train/fainted/fall065.jpg removed
./data/train/fainted/fall066.jpg removed
./data/train/fainted/fall069.jpg removed
./data/train/fainted/fall070.jpg removed
./data/train/fainted/fall072.jpg removed
./data/train/fai